# Adventures in Zero-Shot Text Classification

Transfer learning has had an enormous impact in Natural Language Processing. Thanks to models like BERT, it is now possible to train more accurate NLP models than before, and typically do so with less labeled data. Now that finetuning language models has become the standard procedure in NLP, it’s only natural to get curious and ask: do we need any task-specific labeled training items at all?

In this notebook, we investigate two available models for zero-shot text classification and evaluate how they perform. The code for this article is available in [our repository of NLP notebooks](https://nlptown.github.io/nlp-notebooks/).

## Zero-shot text classification

Zero-shot and few-shot NLP models take transfer learning to the extreme: their goal is to make predictions for an NLP task without having seen one single labeled item (for zero-shot learning), or very few such items (for few-shot learning) specific to that task. The most well-known example is doubtlessly [OpenAI’s GPT-3](https://arxiv.org/abs/2005.14165), which has proved to be a very successful few-shot learner for a wide range of applications. While running GPT-3 lies beyond the means of most developers, luckily several smaller alternatives are available.

In 2020, [Flair](https://github.com/flairNLP/flair) and [Transformers](https://huggingface.co/transformers/), two of the most popular NLP libraries, both added zero-shot classification to their offering. Flair, on the one hand, makes use of a so-called TARS classifier, short for [Text-Aware Representation of Sentences](https://kishaloyhalder.github.io/pdfs/tars_coling2020.pdf), which can be run with just a few lines of code:

```
classifier = TARSClassifier.load('tars-base')
sentence = Sentence('Your example text')
classifier.predict_zero_shot(sentence, [label1, label2, …])
```

Transformers, on the other hand, makes it possible to use a range of models from the [Hugging Face model hub](https://huggingface.co/models) in their `zero-shot-classification` pipeline:

```
classifier = pipeline('zero-shot-classification', model="your-nli-model", device=0)
classifier('Your example text', [label1, label2, …])
```

Despite the obvious similarities, the two implemented classifiers approach zero-shot text classification quite differently.

The zero-shot pipeline in the Transformers library treats text classification as natural language inference (NLI). This approach was pioneered by [Yin et al. in 2019](https://arxiv.org/abs/1909.00161). In NLI, a model takes two sentences as input &mdash; a premise and a hypothesis &mdash; and decides whether the hypothesis follows from the premise (`entailment`), contradicts it (`contradiction`), or neither (`neutral`). For example, the premise _David killed Goliath_ entails the hypothesis _Goliath is dead_, is contradicted by _Goliath is alive_ and doesn’t allow us to draw any conclusions about _Goliath is a giant_. This NLI template can be reused for text classification by taking the text we’d like to label as the premise, and rephrasing every candidate class as a hypothesis. For a task such as polarity classification, the premise could be an opinion like _I loved this movie_, with the hypotheses _This sentence is positive_, _This sentence is negative_ or _This sentence is neutral_. The classifier will then determine the relationship between the premise and every hypothesis. In single-label classification, all resulting `entailment` scores are softmaxed to identify the single most probable class; in multi-label classification, the scores for `entailment` and `contradiction` are softmaxed for every label independently, so that several relevant labels can be identified.

The TARS classifier in the Flair library takes a different course. Similar to the previous approach, it abstracts away from the specificities of individual classification tasks by feeding both the label and the text as input to a BERT classifier, separated by the `[SEP]` token. The main difference lies in the fact that this BERT model is not finetuned for NLI, but for a generic version of text classification. This is done by training the model to label every input pair as either true or false. To make sure it can handle a variety of classification tasks, Flair’s TARS classifier is finetuned on nine different datasets, with applications ranging from polarity to topic classification. For single-label classification, only the class with the highest score for `True` is selected as the final prediction; for multi-label classification, all classes with the prediction `True` are returned.

Although both approaches to zero-shot classification sound very attractive, they share one disadvantage: in contrast to traditional text classification, each input text requires several forward passes through the model &mdash; one for each candidate label. The models are therefore less computationally efficient than more traditional text classifiers. Still, if they can bypass the need for expensive data labeling, for many applications this may be a small price to pay.

## Data

The more pressing question is therefore how well zero-shot text classification works exactly. To find out, we evaluated the classifiers above on five different tasks, from topic classification to sentiment analysis. We used four datasets that are all available from the [Hugging Face datasets hub](https://huggingface.co/datasets),
making sure that none of these datasets was used to finetune Flair’s TARS classifier. From each we selected 1,000 random test items:

- [yahoo_answers_topics](https://huggingface.co/datasets/yahoo_answers_topics): questions and answers from Yahoo Answers, classified into 10 topics, such as `Society & Culture` and `Science & Mathematics`. As the input to the model we use the best answer only (without the question).
- [banking 77](https://huggingface.co/datasets/banking77): a set of online user queries from the banking domain, each labeled with one of 77 intents. This is a challenging dataset, as the intents (such as `card_about_to_expire` and `card_not_working`) are very fine-grained.
- [tweet_eval](https://huggingface.co/datasets/tweet_eval): English tweets labeled for a variety of tasks. We tested if the models can predict the emotion &mdash; `anger`, `joy`, `optimism` or `sadness` &mdash; and the sentiment polarity of the tweets &mdash; `positive`, `negative` or `neutral`.
- [financial_phrasebank](https://huggingface.co/datasets/financial_phrasebank): financial news sentences (such as _Sales have risen in other export markets_) with a polarity label &mdash; `positive`, `negative` or `neutral`. We only selected sentences for which all annotators agreed on the label.

In [ ]:
from datasets import load_dataset

yahoo = load_dataset('yahoo_answers_topics')
banking = load_dataset('banking77')
amazon = load_dataset('amazon_reviews_multi')
financial = load_dataset('financial_phrasebank', 'sentences_allagree')
tweets = load_dataset('tweet_eval', 'sentiment')
tweets_emotion = load_dataset('tweet_eval', 'emotion')

In [ ]:
import re

def clean(label):
    label = re.sub("([a-z])([A-Z])", "\\1 \\2", label)
    label = label.replace("_", " ")
    return label

In [ ]:
import random

random.seed(1)

def sample_test_data(texts,  labels, size):
    
    data = list(zip(texts, labels))
    data = [item for item in data if len(item[0].strip()) > 0]
    
    random.shuffle(data)

    texts, labels = zip(*data)
    
    return texts[:size], labels[:size], texts[size:], labels[size:]


In [ ]:
yahoo_test_texts, yahoo_test_labels, _, _ = sample_test_data(yahoo['test']['best_answer'], yahoo['test']['topic'], 1000)
banking_test_texts, banking_test_labels, _, _ = sample_test_data(banking['test']['text'], banking['test']['label'], 1000)
financial_test_texts, financial_test_labels, financial_train_texts, financial_train_labels = sample_test_data(financial['train']['sentence'], financial['train']['label'], 1000)
tweets_test_texts, tweets_test_labels, _, _ = sample_test_data(tweets['test']['text'], tweets['test']['label'], 1000)
tweets_emotion_test_texts, tweets_emotion_test_labels, _, _ = sample_test_data(tweets_emotion['test']['text'], tweets_emotion['test']['label'], 1000)

In [ ]:
data = [
    {
        "name": "yahoo_answers_topics", 
        "test_texts": yahoo_test_texts, 
        "test_labels": yahoo_test_labels, 
        "train_texts": yahoo['train']['best_answer'], 
        "train_labels": yahoo['train']['topic'], 
        "class_names": [clean(label) for label in yahoo['test'].features['topic'].names]
    },
    {
        "name": "banking77", 
        "test_texts": banking_test_texts,
        "test_labels": banking_test_labels, 
        "train_texts": banking['train']['text'], 
        "train_labels": banking['train']['label'], 
        "class_names": [clean(label) for label in banking['test'].features['label'].names]
    },    
    {
        "name": "tweet_eval: emotion", 
        "test_texts": tweets_emotion_test_texts, 
        "test_labels": tweets_emotion_test_labels, 
        "train_texts": tweets_emotion['train']['text'], 
        "train_labels": tweets_emotion['train']['label'], 
        "class_names": [clean(label) for label in tweets_emotion['test'].features['label'].names]
    },
    {
        "name": "tweet_eval: sentiment", 
        "test_texts": tweets_test_texts, 
        "test_labels": tweets_test_labels,
        "train_texts": tweets['train']['text'], 
        "train_labels": tweets['train']['label'],
        "class_names": [clean(label) for label in tweets['test'].features['label'].names]
    },
    {
        "name": "financial_phrasebank", 
        "test_texts": financial_test_texts, 
        "test_labels": financial_test_labels, 
        "train_texts": financial_train_texts, 
        "train_labels": financial_train_labels, 
        "class_names":  [clean(label) for label in financial['train'].features['label'].names]
    }    
]

## The models

We used three different zero-shot text classifiers in our tests: Flair’s `TARSClassifier`, and two Transformers models finetuned for NLI: `bart-large-mnli` and `roberta-large-mnli`. The graph below shows their accuracies on the five tasks. The results are as varied as the datasets, but one pattern is immediately clear: the best model always takes the NLI approach. For the Yahoo Answers topics, Bart gives the best accuracy (39.2%), followed by TARS and Roberta, which both obtain 27.5%. Although the banking task appears much more challenging at first sight, the NLI models perform even better here: they both classify over 41% of the test items correctly, leaving the TARS classifier far behind. On four-way tweet emotion classification, both Bart (73.6%) and Roberta (71.9%) perform surprisingly well and easily beat TARS (32.3%).

The two polarity tasks deserve some additional explanation. Because our first evaluation run showed very low scores for TARS (accuracies below the random baseline of 33%), we took a closer look at the results and found that in most cases, TARS failed to predict a single label for the news sentences and tweets. To fix this, we performed a second run, where we made TARS return `neutral` for every sentence without a label. It is those scores you see below.
For both polarity tasks, Roberta gives the best results, with 54.0% accuracy for the tweets and 58.8% for the
financial news sentences. TARS and Bart obtain a similar result on the tweets, with 48.2% and 49.0% accuracy, respectively. On the financial news sentences, TARS is better, with 51.7% against 38.8%.

In [ ]:
from flair.models.text_classification_model import TARSClassifier
from flair.data import Sentence
from tqdm.notebook import tqdm

def evaluate_flair(dataset, default_name='neutral'):

    classifier = TARSClassifier.load('tars-base')
        
    total, correct = 0, 0
    for item, gold_label_idx in tqdm(zip(dataset["test_texts"], dataset["test_labels"]), total = len(dataset["test_texts"])):
        sentence = Sentence(item)
        classifier.predict_zero_shot(sentence, dataset["class_names"])
        sorted_labels = sorted(sentence.to_dict()['labels'], key=lambda k: k['confidence'], reverse=True)
        gold_label = dataset["class_names"][gold_label_idx]
        if len(sorted_labels) > 0:
            predicted_label = sorted_labels[0]['value']
        else:
            predicted_label = default_name
        if predicted_label == gold_label:
            correct += 1
        total += 1
        
    return correct/total

In [ ]:
import numpy as np
from transformers import pipeline

def evaluate_huggingface(dataset, template=None, model='base'):

    if model == 'base':
        classifier = pipeline("zero-shot-classification", device=0)
    else:
        classifier = pipeline("zero-shot-classification", model="roberta-large-mnli", device=0)
    
    correct = 0
    predictions, gold_labels = [], []
    for text, gold_label_idx in tqdm(zip(dataset["test_texts"], dataset["test_labels"]), total=len(dataset["test_texts"])):

        if template is not None:
            result = classifier(text, dataset["class_names"], multi_label=False, template=template)
        else:
            result = classifier(text, dataset["class_names"], multi_label=False)
        predicted_label = result['labels'][0]
        
        gold_label = dataset["class_names"][gold_label_idx]
        
        predictions.append(predicted_label)
        gold_labels.append(gold_label)
        
        if predicted_label == gold_label:
            correct += 1
            
    accuracy = correct/len(predictions)
    return accuracy


## Evaluation

In [ ]:
results = {}

for dataset in data:
    print(dataset['name'])
    results[dataset['name']] = {}
    
    flair_acc = evaluate_flair(dataset)
    results[dataset['name']]['Flair TARS'] = flair_acc
    print("Flair:", flair_acc)
    
    huggingface_acc = evaluate_huggingface(dataset)
    results[dataset['name']]['Transformers Bart'] = huggingface_acc
    print("Huggingface Bart", huggingface_acc)

    huggingface_acc_roberta = evaluate_huggingface(dataset, model='roberta')
    results[dataset['name']]['Transformers Roberta'] = huggingface_acc_roberta
    print("Huggingface Roberta", huggingface_acc_roberta)
    


In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(results)
df = df.transpose()
df.plot(kind='bar', figsize=(12,7), colormap='copper', rot=0)

## The impact of class names

In traditional supervised learning, the actual names of the labels do not have any impact on the performance of the model. You’re free to call your classes whatever you want &mdash; `positive`, `politics` or `aardvark`, it makes no difference at all. In zero-shot text classification, these names suddenly become important. Both the NLI and TARS classifiers add the label itself to the input of the model, so that the class names have the power to change the predictions. In general, the more semantic information about the class they contain, and the more similar they are to the type of data that the model was finetuned on, the better we can expect the classifier to perform.

Let’s take polarity classification as an example. While `positive`, `neutral` and `negative` are the traditional class names for this task, they may not be optimal for a zero-shot approach. To test this out, we experimented
with two alternative sets of names for the financial news data: `good news`, `neutral news` and `bad news` on the one hand, and `happy news`, `neutral news` and `unhappy news` on the other. As you can see in the figure below, this has a very positive effect on the accuracy of the classifiers. Both TARS (62.0%) and Bart (61.9%) now perform better than the original Roberta, although Bart only does so with the happy/unhappy class names. Roberta itself jumps another 13%, to an accuracy of over 73% with both alternative sets of names. Zero-shot classifiers may reduce the need for labeling, but they do introduce the necessary task of searching for good class names.

In [ ]:
print(data[3]["class_names"])

In [ ]:
alternative_results = {}
alternative_results['positive/negative'] = results['financial_phrasebank']

alternative_financial_news = {
    "name": "good/bad", 
    "test_texts": financial_test_texts, 
    "test_labels": financial_test_labels, 
    "class_names":  ["bad news", "neutral news", "good news"]
    }

alternative_results[alternative_financial_news['name']] = {}
flair_acc = evaluate_flair(alternative_financial_news, default_name='neutral news')
alternative_results[alternative_financial_news['name']]['Flair TARS'] = flair_acc
print("Flair:", flair_acc)

huggingface_acc = evaluate_huggingface(alternative_financial_news)
alternative_results[alternative_financial_news['name']]['Transformers Bart'] = huggingface_acc
print("Huggingface Bart", huggingface_acc)

huggingface_acc_roberta = evaluate_huggingface(alternative_financial_news, model='roberta')
alternative_results[alternative_financial_news['name']]['Transformers Roberta'] = huggingface_acc_roberta
print("Huggingface Roberta", huggingface_acc_roberta)


In [ ]:
alternative_financial_news = {
    "name": "happy/unhappy", 
    "test_texts": financial_test_texts, 
    "test_labels": financial_test_labels, 
    "class_names":  ["unhappy news", "neutral news", "happy news"]
    }

alternative_results[alternative_financial_news['name']] = {}
flair_acc = evaluate_flair(alternative_financial_news, default_name='neutral news')
alternative_results[alternative_financial_news['name']]['Flair TARS'] = flair_acc
print("Flair:", flair_acc)

huggingface_acc = evaluate_huggingface(alternative_financial_news)
alternative_results[alternative_financial_news['name']]['Transformers Bart'] = huggingface_acc
print("Huggingface Bart", huggingface_acc)

huggingface_acc_roberta = evaluate_huggingface(alternative_financial_news, model='roberta')
alternative_results[alternative_financial_news['name']]['Transformers Roberta'] = huggingface_acc_roberta
print("Huggingface Roberta", huggingface_acc_roberta)

In [ ]:
df = pd.DataFrame.from_dict(alternative_results)
df.plot(kind='bar', figsize=(12,7), colormap='copper', rot=0)

In [ ]:
df

## Few-shot learning

As we saw with polarity classification, the TARS classifier tends to suffer from low recall. In tasks with a default class, such as `neutral`, this label can serve as a fallback option, but in other cases we need a different solution.
Luckily, Flairs has made it easy to finetune TARS on a handful of training examples. Let’s see what happens if we give the model one example of what we mean by each class, and finetune it on this small training set. Because the performance of the final model will depend on what training instances we pick, we repeat this process ten times and always select random examples from the training corpus.

The figure below shows that the TARS classifier benefits greatly from this few-shot learning procedure. The impact is clearest for the three tasks without a default class: TARS’s accuracy jumps significantly, and in two out of three cases it becomes competitive with the best NLI model. Interestingly, this even happens for the emotion dataset, where we’ve used just four examples as our training set. For the polarity tasks, the benefit is less clear, as we already fixed the recall problem by introducing a default class, and only worked with three labeled examples for few-shot learning.

In [ ]:
from flair.data import Corpus
from flair.datasets import SentenceDataset

def sample_training_corpus(dataset, seed):

    random.seed(seed)
    
    seen_labels = set()
    sentences = []

    train_set = list(zip(dataset['train_texts'], dataset['train_labels']))
    random.shuffle(train_set)
    
    for text, label in train_set:
        topic = dataset['class_names'][label]
        if topic not in seen_labels:
            sentences.append(Sentence(text).add_label("_or_".join(dataset['class_names']), topic))
            seen_labels.add(topic)
        if len(seen_labels) == len(dataset['class_names']):
            break

    train = SentenceDataset(sentences)
    corpus = Corpus(train=train)
    return corpus

In [ ]:
from flair.trainers import ModelTrainer

def train(corpus, dataset):

    tars = TARSClassifier.load('tars-base')
    tars.add_and_switch_to_new_task(dataset['name'], label_dictionary=corpus.make_label_dictionary())

    trainer = ModelTrainer(tars, corpus)

    trainer.train(base_path='/tmp/' + dataset['name'], # path to store the model artifacts
              learning_rate=0.02, # use very small learning rate
              mini_batch_size=1, # small mini-batch size since corpus is tiny
              max_epochs=10, # terminate after 10 epochs
              train_with_dev=False,
              )

In [ ]:
def evaluate(dataset, default_name='neutral'):
    tars = TARSClassifier.load(f'/tmp/{dataset["name"]}/final-model.pt')

    total, correct = 0, 0
    for item, gold_label_idx in tqdm(zip(dataset["test_texts"], dataset["test_labels"]), total = len(dataset["test_texts"])):
        sentence = Sentence(item)
        tars.predict(sentence)
        sorted_labels = sorted(sentence.to_dict()['labels'], key=lambda k: k['confidence'], reverse=True)
        
        gold_label = dataset["class_names"][gold_label_idx]
        if len(sorted_labels) > 0:
            predicted_label = sorted_labels[0]['value']
        else:
            predicted_label = default_name                

        if predicted_label == gold_label:
            correct += 1

        total += 1

    return correct/total

In [ ]:
#results_fewshot = {}

for dataset in data[3:]:
    results_fewshot[dataset['name']] = []
    for seed in range(10):

        corpus = sample_training_corpus(dataset, seed)
        train(corpus, dataset)
        acc = evaluate(dataset)
        results_fewshot[dataset['name']].append(acc)

        print("Accuracy:", acc)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import csv

df = pd.DataFrame(results_fewshot)
#sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(12, 7))

# Show each distribution with both violins and points
sns.violinplot(data=df, palette="Set3", inner="points",bw =.2, cut=2,linewidth=2, orient="v")

sns.despine(left=True)

#f.suptitle("TARS Few-Shot Learning", fontsize=18, fontweight='bold')
ax.set_ylabel("Accuracy", size = 12, alpha=0.7)
ax.set_ylim([0, 1])
#ax.set_ylabel("Model",size = 14, alpha=0.7)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(11) 

    
for i, result in enumerate(results):
    ax.plot(i, results[data[i]['name']]['Flair TARS'], 'rx', markersize=12)
    ax.plot(i, results[data[i]['name']]['Transformers Bart'], 'r+', markersize=12)
    ax.plot(i, results[data[i]['name']]['Transformers Roberta'], 'ro', markersize=12)
    
ax.legend(['Flair TARS zero-shot', 'Transformers Bart', 'Transformers Roberta'], fontsize=12)

## Conclusions

Without a doubt, zero-shot learning is an extraordinary application of transfer learning. Zero-shot classifiers predict the class of a text without having seen a single labeled example, and in some cases do so with a higher accuracy than supervised models that have been trained on hundreds of labeled training items. Their success is far from guaranteed &mdash; it depends on the particular task and a careful selection of class names &mdash; but in the right circumstances, these models can get you a long way towards accurate text classification.

At the same time, the open-source zero-shot classifiers we tested out are no magic solutions, as it’s unlikely they are going to give you optimal performance on a specialized NLP task. For such applications, manually labeling a large number of examples, for example with a tool like <a href='https://www.tagalog.ai/'>Tagalog</a> still gives you the best chance of success. Even in those cases, however, zero-shot classification can prove useful, for example as a way to speed up manual labeling by suggesting potentially relevant labels to the annotators. It’s clear zero-shot and few-shot classification is here to stay, and can be a useful tool in any NLPer’s toolkit.